In [1]:
import sys

import pymgrid.MicrogridGenerator as mg
import pymgrid.utils.DataGenerator as dg
import pymgrid.algos.Control as control
import pickle
import importlib
import pandas as pd

def reload():
    importlib.reload(dg)
    importlib.reload(mg)
    importlib.reload(control)
reload()  

In [2]:
env = mg.MicrogridGenerator(nb_microgrid=25)
env.generate_microgrid()


load  cost_loss_load  cost_overgeneration  cost_co2  PV_rated_power  \
0     960              10                    1       0.1          969.60   
1   60363              10                    1       0.1        30785.13   
2   43725              10                    1       0.1        54219.00   
3   78147              10                    1       0.1        38292.03   
4   49813              10                    1       0.1        73225.11   
5   28560              10                    1       0.1        30559.20   
6   49216              10                    1       0.1        53645.44   
7   11798              10                    1       0.1        10028.30   
8   64065              10                    1       0.1        34595.10   
9   98996              10                    1       0.1       136614.48   
10  60144              10                    1       0.1        59542.56   
11  27615              10                    1       0.1        29548.05   
12  48214              10                    1       0.1        63642.48   
13  19941              10                    1       0.1         8774.04   
14  98643              10                    1       0.1        82860.12   
15  17296              10                    1       0.1        16604.16   
16  48622              10                    1       0.1        70015.68   
17  31257              10                    1       0.1        15628.50   
18  34418              10                    1       0.1        11702.12   
19   1529              10                    1       0.1          565.73   
20  79186              10                    1       0.1       105317.38   
21  95435              10                    1       0.1       136472.05   
22  80372              10                    1       0.1       111717.08   
23   9453              10                    1       0.1         9169.41   
24  50157              10                    1       0.1        36614.61   

    battery_soc_0  battery_power_charge  battery_power_discharge  \
0        0.200000                   363                      363   
1        0.200000                 22819                    22819   
2        0.200000                 16529                    16529   
3        0.200000                 25613                    25613   
4        0.200000                 16327                    16327   
5        1.000000                 14396                    14396   
6        0.960305                 44844                    44844   
7        1.000000                  4834                     4834   
8        0.200000                 26396                    26396   
9        0.537479                 49897                    49897   
10       0.200000                 54801                    54801   
11       1.000000                 17399                    17399   
12       0.200000                 35932                    35932   
13       0.200000                 11889                    11889   
14       0.200000                 40643                    40643   
15       1.000000                  6539                     6539   
16       0.534119                 15025                    15025   
17       0.200000                  9659                     9659   
18       1.000000                 20520                    20520   
19       0.200000                   627                      627   
20       0.200000                 32442                    32442   
21       0.200000                 42674                    42674   
22       0.200000                 73231                    73231   
23       0.200000                  3574                     3574   
24       0.200000                 25832                    25832   

    battery_capacity  battery_efficiency  battery_soc_min  battery_soc_max  \
0               1452                 0.9              0.2                1   
1              91274                 0.9              0.2                1   
2              66116     

In [ ]:
comparison_list = []
forecast_accuracies = [50, 70, 85]
percentiles = [0.25, 0.5, 0.75]

sampling_args = dict()
sampling_args['push_individual_vals'] = True # For pv data
sampling_args['push_individual_ratio'] = 0.5 # For pv data

total_iter = len(env.microgrids)*len(forecast_accuracies)*len(percentiles)

checkpoint = pd.DataFrame()
checkpoint = pd.read_csv('checkpoint.csv')

j0 = checkpoint['j'].values[0]
k0 = checkpoint['k'].values[0]
l0 = checkpoint['l'].values[0]
check = True

for j, microgrid in enumerate(env.microgrids):
    for k, accuracy in enumerate(forecast_accuracies):
        for l, percentile in enumerate(percentiles):
            
#             if j0 == j and k0==k and l0==l:
#                 check = True
#                 continue
            
            if check == False:
                continue
            
            
            m = (j*len(forecast_accuracies)+k)*len(percentiles)+l
            print(round(100*m/total_iter,2),'%')
        
            SAA = control.SampleAverageApproximation(microgrid, preset_to_use=accuracy)
            mape = SAA.validate_forecasts(forecasts = SAA.forecasts, aggregate = True)
            
            mpc_output = SAA.run_deterministic_on_forecast()
            saa_output = SAA.run(optimal_percentile = percentile, use_previous_samples=False, **sampling_args)
            
            d = dict(microgrid=microgrid, percentile=percentile, accuracy=accuracy, mpc_output=mpc_output, saa_output = saa_output)
            #comparison_list.append(d)
            
            checkpoint = pd.DataFrame({'j':j, 'k':k, 'l':l}, index=[0])
            checkpoint.to_csv('checkpoint.csv')
            
#             filename = 'sensitivity_1026/comparison_algos_'+str(j)+'_'+str(k)+'_'+str(l)
#             outfile = open(filename,'wb')
#             pickle.dump(d,outfile)
#             outfile.close()
            saa_cost = sum(saa_output['cost']['cost'][5846:])
            mpc_cost = sum(mpc_output['cost']['cost'][5846:])

            print (f"Microgrid {j} | Accuracy {accuracy} | Percentile {percentile}")
            print (f" SAA {saa_cost} | MPC {mpc_cost}" )


0.0 %
Using preset forecast arguments


/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py:212: RuntimeWarning:

invalid value encountered in true_divide



Creating sample 0
Creating sample 1
Creating sample 2
Creating sample 3
Creating sample 4
Creating sample 5
Creating sample 6
Creating sample 7
Creating sample 8
Creating sample 9
Microgrid 0 | Accuracy 50 | Percentile 0.25
 SAA 351141.2962738597 | MPC 0
0.44 %
Using preset forecast arguments


/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py:212: RuntimeWarning:

invalid value encountered in true_divide



Creating sample 0
Creating sample 1
Creating sample 2
Creating sample 3
Creating sample 4
Creating sample 5
Creating sample 6
Creating sample 7
Creating sample 8
Creating sample 9
Microgrid 0 | Accuracy 50 | Percentile 0.5
 SAA 350694.9977374617 | MPC 0
0.89 %
Using preset forecast arguments


...

Creating sample 0
Creating sample 1
Creating sample 2
Creating sample 3
Creating sample 4
Creating sample 5
Creating sample 6
Creating sample 7
Creating sample 8
Creating sample 9


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-a73c20890801>", line 38, in <module>
    saa_output = SAA.run(optimal_percentile = percentile, use_previous_samples=False, **sampling_args)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 314, in run
    optimal_percentile=optimal_percentile, verbose=verbose)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 380, in run_mpc_on_group
    horizon_output = self.mpc.mpc_single_step(sample, output, j)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 1441, in mpc_single_step
    sample.loc[current_step:current_step + horizon - 1, 'pv'].values, temp_grid, price_import,
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-a73c20890801>", line 38, in <module>
    saa_output = SAA.run(optimal_percentile = percentile, use_previous_samples=False, **sampling_args)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 314, in run
    optimal_percentile=optimal_percentile, verbose=verbose)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 380, in run_mpc_on_group
    horizon_output = self.mpc.mpc_single_step(sample, output, j)
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site-packages/pymgrid/algos/Control.py", line 1441, in mpc_single_step
    sample.loc[current_step:current_step + horizon - 1, 'pv'].values, temp_grid, price_import,
  File "/opt/anaconda3/envs/ploutos/lib/python3.7/site

In [ ]:
sum(mpc_output['cost']['cost'])



In [1]:
saa_output

NameError: name 'saa_output' is not defined

In [ ]:
filename = 'comparison_algos'
outfile = open(filename,'wb')
pickle.dump(comparison_list,outfile)
outfile.close()

In [14]:
checkpoint = {'j':j,
              'k':k,
              'l':l}

pd.DataFrame({'j':j,
              'k':k,
              'l':l}, index=[0])

j  k  l
0  0  0  0

In [17]:
percentile


0.25